In [1]:
year = 2023
day = 21

In [2]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)


# Z and n_steps_explore are empiracally found for true data and example data
# real input
data = puzzle.input_data
Z = 1
n_steps_explore = 400

# # example input
# data = puzzle.examples[0].input_data
# Z = 5
# n_steps_explore = 200

data = data.strip()
data = data.split("\n")
data = [list(d) for d in data]
data = np.array(data)
start = (np.where(data == "S")[0][0], np.where(data == "S")[1][0])
data[np.where(data == "S")] = "."

current_day is only available in December (EST)


In [3]:
import numpy as np

np.set_printoptions(edgeitems=30, linewidth=100000,
                    formatter=dict(float=lambda x: "%s" % x))

In [4]:
possible_spots = {start}

all_possible_spots = []

steps = []

H, W = data.shape

bbox = [0, 0, 0, 0]

# brute force for part A / explore for part B
for i in range(1, n_steps_explore):
    new_possible_spots = set()
    for y, x in possible_spots:
        for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            y_, x_ = (y + dy), (x + dx)
            if data[y_ % H, x_ % W] == ".":
                new_possible_spots.add((y_, x_))
    possible_spots = new_possible_spots
    print(i, len(possible_spots))
    steps.append(len(possible_spots))
    all_possible_spots.append(list(possible_spots))


1 4
2 9
3 15
4 23
5 33
6 47
7 57
8 77
9 86
10 114
11 127
12 161
13 177
14 215
15 233
16 269
17 293
18 332
19 362
20 403
21 437
22 482
23 522
24 570
25 613
26 665
27 708
28 759
29 810
30 866
31 926
32 984
33 1048
34 1111
35 1176
36 1241
37 1309
38 1372
39 1442
40 1521
41 1591
42 1675
43 1748
44 1836
45 1904
46 1992
47 2067
48 2166
49 2239
50 2340
51 2414
52 2527
53 2601
54 2720
55 2799
56 2920
57 2999
58 3139
59 3205
60 3348
61 3423
62 3596
63 3681
64 3858
65 3943
66 4122
67 4211
68 4394
69 4461
70 4638
71 4723
72 4896
73 4988
74 5174
75 5270
76 5444
77 5552
78 5723
79 5837
80 6019
81 6134
82 6316
83 6430
84 6624
85 6740
86 6926
87 7053
88 7251
89 7382
90 7574
91 7715
92 7905
93 8049
94 8242
95 8383
96 8590
97 8733
98 8934
99 9080
100 9288
101 9442
102 9649
103 9811
104 10018
105 10199
106 10405
107 10587
108 10798
109 10973
110 11188
111 11365
112 11599
113 11778
114 11995
115 12182
116 12413
117 12606
118 12830
119 13030
120 13269
121 13481
122 13704
123 13927
124 14157
125 14380
126 

In [5]:
submit(steps[64-1], part="a", day=day, year=year)

Part a already solved with same answer: 3858


In [6]:
box_filter = lambda by, bx: lambda loc: (loc[0] >= 0+by*H) and (loc[0] < H+by*H) and (loc[1] >= 0+bx*W) and (loc[1] < W+bx*W)

repeat_value = np.array([len(list(filter(box_filter(0, 0), l))) for l in all_possible_spots]).max()
# each box will repeat between two possible values (checkerboard-like) when it's satured, so we'll just take the max of the two
repeat_value

7780

In [7]:
box_repeats = {}
box_sequences = {}

for by in range(-Z, Z+1):
    for bx in range(-Z, Z+1):
        # filter the known steps count on the boxes that are tiles of the input data, and find the repeat index and repeat sequence for each box.
        # later we'll assume other boxes are just repeats of these boxes, with some offset.
        lens = np.array([len(list(filter(box_filter(by, bx), l))) for l in all_possible_spots])
        repeat_index = np.where(lens == repeat_value)[0][0]
        print(by, bx, repeat_index)
        box_repeats[(by, bx)] = repeat_index
        box_sequences[(by, bx)] = lens


-1 -1 391
-1 0 260
-1 1 391
0 -1 260
0 0 129
0 1 260
1 -1 391
1 0 260
1 1 391


In [8]:
start_offset = 35
outputs = []
for i in range(1, 6):
    outputs.append(list(sorted(map(lambda l: (l[0]%H, l[1]%W), filter(box_filter(i+1, i+1), all_possible_spots[start_offset+i*22])))))

assert outputs[0] == outputs[1]
assert outputs[1] == outputs[2]
assert outputs[2] == outputs[3]
assert outputs[3] == outputs[4]

In [9]:
Z_ref = Z
# for boxes further away, they will show the same pattern as the closest box, but with an offset.
# this function finds the closest box, and the offset.
def find_ref_box(by, bx):
    ref_by = int(min(abs(by), Z_ref)* (by/abs(by))) if by != 0 else 0
    ref_bx = int(min(abs(bx), Z_ref)* (bx/abs(bx))) if bx != 0 else 0
    offset_x = abs(by) - abs(ref_by) if abs(by) > abs(ref_by) else 0
    offset_y = abs(bx) - abs(ref_bx) if abs(bx) > abs(ref_bx) else 0
    return ref_by, ref_bx, offset_x, offset_y

find_ref_box(10, 1)

(1, 1, 9, 0)

In [10]:
# finds the amount of steps after which box/tile (by, bx) will repeat
def get_box_repeat(by, bx):
    ref_by, ref_bx, offset_x, offset_y = find_ref_box(by, bx)
    return box_repeats[(ref_by, ref_bx)]+H*offset_y+W*offset_x

# check that it works. This only makes sense when Z > 1 (example data)
for by in range(-Z, Z+1):
    for bx in range(-Z, Z+1):
        ref_by, ref_bx, offset_x, offset_y = find_ref_box(by, bx)
        assert box_repeats[(by, bx)] == get_box_repeat(by, bx)

In [11]:
def get_box_value(by, bx, i):
    ref_by, ref_bx, offset_x, offset_y = find_ref_box(by, bx)

    box_repeat = get_box_repeat(by, bx)
    ref_box_repeat = get_box_repeat(ref_by, ref_bx)
    seq_idx = ref_box_repeat - box_repeat + i
    if seq_idx < 0:
        return 0
    if seq_idx > ref_box_repeat:
        return box_sequences[(ref_by, ref_bx)][ref_box_repeat + (seq_idx-ref_box_repeat) % 2]
    else:
        return box_sequences[(ref_by, ref_bx)][seq_idx]

In [12]:
new_points = []
total_sum = 0

count = 26501365-1
N = count // H - 3

# all the tiles that are not near the fringe of our expanding reach and are just oscillating between two values
# we can just calculate the total sum of these tiles, and add the fringe tiles later
# the (N+1)**2 and (N)**2 are the amount of tiles oscillating between the two values
total_sum = (N+1)**2*get_box_value(N%2,0, count) + (N)**2*get_box_value((N-1)%2,0, count)

# the fringe tiles are the tiles that are not oscillating between two values, but are still in the fringe of our expanding reach
for n in range(N+1, N+10):
    # print(n)
    for a in range(0, n):
        b = n-a
        # print(a,b)
        total_sum += get_box_value(a, b, count)
        total_sum += get_box_value(-b, a, count)
        total_sum += get_box_value(-a, -b, count)
        total_sum += get_box_value(b, -a, count)
        new_points.append((a, b))

total_sum

636350496972143

In [13]:
submit(total_sum, part="b", day=day, year=year)

coerced int64 value 636350496972143 for 2023/21


Part b already solved with same answer: 636350496972143
